In [1]:
#Uploading packages we'll leave later

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
#Read in Migration data
migration_data = pd.read_csv('Raw Data/Persons of Concern/for_map_unhcr_popstats_export_persons_of_concern_all_data.csv', dtype=str)

#Rename data
migration_data = migration_data.rename(columns={'Country / territory of asylum/residence': 'Country', 'Refugees (incl. refugee-like situations)': 'Refugees', 'Asylum-seekers (pending cases)': 'Asylum', 'Internally displaced persons (IDPs)': 'Internally displaced persons'})

In [34]:
#Check it out
migration_data

,Year,Country,Origin,Refugees,Asylum,Returned refugees,Internally displaced persons,Returned IDPs,Stateless persons,Others of concern,Total Population
0,1951,Australia,Various/Unknown,180000,NaN,NaN,NaN,NaN,NaN,NaN,180000
1,1951,Austria,Various/Unknown,282000,NaN,NaN,NaN,NaN,NaN,NaN,282000
2,1951,Belgium,Various/Unknown,55000,NaN,NaN,NaN,NaN,NaN,NaN,55000
3,1951,Canada,Various/Unknown,168511,NaN,NaN,NaN,NaN,NaN,NaN,168511
4,1951,Switzerland,Various/Unknown,10000,NaN,NaN,NaN,NaN,NaN,NaN,10000
5,1951,Germany,Various/Unknown,265000,NaN,NaN,NaN,NaN,NaN,NaN,265000
6,1951,Denmark,Various/Unknown,2000,NaN,NaN,NaN,NaN,NaN,NaN,2000
7,1951,Spain,Various/Unknown,2000,NaN,NaN,NaN,NaN,NaN,NaN,2000
8,1951,France,Various/Unknown,290000,NaN,NaN,NaN,NaN,NaN,NaN,290000
9,1951,United Kingdom,Various/Unknown,208000,NaN,NaN,NaN,NaN,NaN,NaN,208000


In [4]:
#Make a new dataset just for refugees going into Brazil and reset the index of that dataset so it can be a dataframe
refugees_in = migration_data.loc[migration_data['Country'] == 'Brazil']
refugees_in = refugees_in.reset_index(drop=True)

#Note - there are a few values that have an * which denotes an unclear count in that category. 
#In our data, we will replace them with zeros

refugees_in.replace(['*'], [0], inplace=True) 
refugees_in.fillna(0, inplace=True)

#Convert columns to numeric where necessary. 
refugees_in[['Year','Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population']] = refugees_in[['Year','Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population']].apply(pd.to_numeric)

#Export refugees_in to CSV
refugees_in.to_csv('Processed Data/refugees_in.csv')

In [5]:
#Make a new dataset that summarizes in-migration of refugees by year
refugees_in_by_year = refugees_in.groupby(['Year'])['Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population'].sum()
refugees_in_by_year.head()

#Export refugees_out_by_year to CSV
refugees_in_by_year.to_csv('Processed Data/refugees_in_by_year.csv')

In [6]:
#Make a new dataset just for refugees coming out Brazil and reset the index of that dataset
refugees_out = migration_data.loc[migration_data['Origin'] == 'Brazil']
refugees_out = refugees_out.reset_index(drop=True)

#Note - there are a few values that have an * which denotes an unclear count in that category. 
#In our data, we will replace them with zeros

refugees_out.replace(['*'], [0], inplace=True) 
refugees_out.fillna(0, inplace=True)

#Convert columns to numeric where necessary. 
refugees_out[['Year','Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population']] = refugees_out[['Year','Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population']].apply(pd.to_numeric)

#Export refugees_out to CSV
refugees_out.to_csv('Processed Data/refugees_out.csv')


In [7]:
#Make a new dataset that summarizes out-migration of refugees by year
refugees_out_by_year = refugees_out.groupby(['Year'])['Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population'].sum()
refugees_out_by_year.head()

#Export refugees_out_by_year to CSV
refugees_out_by_year.to_csv('Processed Data/refugees_out_by_year.csv')

In [8]:
#Now merge to complete dataset to show net change over time.

refugees_net_by_year = pd.merge(refugees_in_by_year.add_suffix('_in'), refugees_out_by_year.add_suffix('_out'), left_index=True, right_index=True, how='outer')

#Clean to get rid of the NaN - fill them in with zeros!
refugees_net_by_year.fillna(0, inplace=True)
refugees_net_by_year

#Write a function to get net values
out_vars = list(refugees_out_by_year.add_suffix('_out').columns.values)
in_vars = list(refugees_in_by_year.add_suffix('_in').columns.values)
root_vars = list(refugees_in_by_year.columns.values)
    
for i in root_vars:
    refugees_net_by_year[f'{i}_net'] = refugees_net_by_year[f'{i}_out'] - refugees_net_by_year[f'{i}_in']
    
refugees_net_by_year

refugees_net_by_year.to_csv('Processed Data/refugees_net_by_year.csv')

In [9]:
refugees_net_by_year = pd.read_csv('Processed Data/refugees_net_by_year.csv')
refugees_in = pd.read_csv('Processed Data/refugees_in.csv')
refugees_out = pd.read_csv('Processed Data/refugees_out.csv')

In [10]:
#Make a dataset that summarizes the top 3 countries sending refugees to Brazil
refugees_in_top3_sources = refugees_in.loc[refugees_in.groupby('Year')['Total Population'].nlargest(3).index.get_level_values(1)].drop(refugees_in.loc[refugees_in.groupby('Year')['Total Population'].nlargest(3).index.get_level_values(1)].columns[0], axis=1)
refugees_in_top3_sources.to_csv('Processed Data/refugees_in_top3_sources.csv')
refugees_in_top3_sources.head()

,Year,Country,Origin,Refugees,Asylum,Returned refugees,Internally displaced persons,Returned IDPs,Stateless persons,Others of concern,Total Population
0,1969,Brazil,Various/Unknown,40000,0,0,0,0,0,0,40000
1,1970,Brazil,Various/Unknown,40000,0,0,0,0,0,0,40000
2,1971,Brazil,Cuba,37800,0,0,0,0,0,0,37800
3,1971,Brazil,Various/Unknown,200,0,0,0,0,0,0,200
5,1972,Brazil,Various/Unknown,36800,0,0,0,0,0,0,36800


In [11]:
#Summarize the movement for Top 3 by type, just to test what's going on
refugees_in_top3_sources[['Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population']].sum().reset_index().rename(columns={'index':'Category', '0': 'count'})

,Category,0
0,Refugees,556562
1,Asylum,35725
2,Returned refugees,268
3,Internally displaced persons,0
4,Returned IDPs,0
5,Stateless persons,0
6,Others of concern,97330
7,Total Population,689889


In [12]:
#Make the same dataset for the Top 5 countries sending folks to Brazil
refugees_in_top5_sources = refugees_in.loc[refugees_in.groupby('Year')['Total Population'].nlargest(5).index.get_level_values(1)].drop(refugees_in.loc[refugees_in.groupby('Year')['Total Population'].nlargest(5).index.get_level_values(1)].columns[0], axis=1)
refugees_in_top5_sources.to_csv('Processed Data/refugees_in_top5_sources.csv')
refugees_in_top5_sources.head()

,Year,Country,Origin,Refugees,Asylum,Returned refugees,Internally displaced persons,Returned IDPs,Stateless persons,Others of concern,Total Population
0,1969,Brazil,Various/Unknown,40000,0,0,0,0,0,0,40000
1,1970,Brazil,Various/Unknown,40000,0,0,0,0,0,0,40000
2,1971,Brazil,Cuba,37800,0,0,0,0,0,0,37800
3,1971,Brazil,Various/Unknown,200,0,0,0,0,0,0,200
5,1972,Brazil,Various/Unknown,36800,0,0,0,0,0,0,36800


In [13]:
#Make a summary table for the Top 5 countries sending folks to Brazil
refugees_in_top5_sources[['Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population']].sum().reset_index().rename(columns={'index':'Category', '0': 'count'})

,Category,0
0,Refugees,571892
1,Asylum,44502
2,Returned refugees,475
3,Internally displaced persons,0
4,Returned IDPs,0
5,Stateless persons,0
6,Others of concern,97330
7,Total Population,714203


In [14]:
#Make a dataset that summarizes the top 3 countries Brazilian refugees go to
refugees_out_top3_sources = refugees_out.loc[refugees_out.groupby('Year')['Total Population'].nlargest(3).index.get_level_values(1)].drop(refugees_out.loc[refugees_out.groupby('Year')['Total Population'].nlargest(3).index.get_level_values(1)].columns[0], axis=1)
refugees_out_top3_sources.to_csv('Processed Data/refugees_out_top3_sources.csv')
refugees_out_top3_sources.head()

,Year,Country,Origin,Refugees,Asylum,Returned refugees,Internally displaced persons,Returned IDPs,Stateless persons,Others of concern,Total Population
0,1971,Chile,Brazil,2000,0,0,0,0,0,0,2000
1,1972,Chile,Brazil,2000,0,0,0,0,0,0,2000
2,1980,Italy,Brazil,5,0,0,0,0,0,0,5
3,1981,Italy,Brazil,5,0,0,0,0,0,0,5
4,1982,Italy,Brazil,5,0,0,0,0,0,0,5


In [15]:
#Make a dataset that summarizes the top 5 countries Brazilian refugees go to
refugees_out_top5_sources = refugees_out.loc[refugees_out.groupby('Year')['Total Population'].nlargest(5).index.get_level_values(1)].drop(refugees_out.loc[refugees_out.groupby('Year')['Total Population'].nlargest(5).index.get_level_values(1)].columns[0], axis=1)
refugees_out_top5_sources.to_csv('Processed Data/refugees_out_top5_sources.csv')
refugees_out_top5_sources.head()

,Year,Country,Origin,Refugees,Asylum,Returned refugees,Internally displaced persons,Returned IDPs,Stateless persons,Others of concern,Total Population
0,1971,Chile,Brazil,2000,0,0,0,0,0,0,2000
1,1972,Chile,Brazil,2000,0,0,0,0,0,0,2000
2,1980,Italy,Brazil,5,0,0,0,0,0,0,5
3,1981,Italy,Brazil,5,0,0,0,0,0,0,5
4,1982,Italy,Brazil,5,0,0,0,0,0,0,5


In [16]:
#Summarize the movement for Top 3 by type, just to test what's going on
refugees_out_top3_sources[['Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population']].sum().reset_index().rename(columns={'index':'Category', '0': 'count'})

,Category,0
0,Refugees,17444
1,Asylum,14537
2,Returned refugees,0
3,Internally displaced persons,0
4,Returned IDPs,0
5,Stateless persons,91
6,Others of concern,0
7,Total Population,32072


In [17]:
#Summarize the movement for Top 5 by type, just to test what's going on
refugees_out_top5_sources[['Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population']].sum().reset_index().rename(columns={'index':'Category', '0': 'count'})

,Category,0
0,Refugees,17816
1,Asylum,14854
2,Returned refugees,9
3,Internally displaced persons,0
4,Returned IDPs,0
5,Stateless persons,197
6,Others of concern,0
7,Total Population,32876


In [18]:
refugees_in_top3_sources['Origin'].value_counts().reset_index().rename(columns={'index':'Country of Origin', 'Origin':'Years of Top 3 Status'}).to_csv('Processed Data/refugees_in_top3_sources_count.csv')
refugees_in_top3_sources['Origin'].value_counts().reset_index().rename(columns={'index':'Country of Origin', 'Origin':'Years of Top 3 Status'})

refugees_in_top5_sources['Origin'].value_counts().reset_index().rename(columns={'index':'Country of Origin', 'Origin':'Years of Top 5 Status'}).to_csv('Processed Data/refugees_in_top5_sources_count.csv')
refugees_in_top5_sources['Origin'].value_counts().reset_index().rename(columns={'index':'Country of Origin', 'Origin':'Years of Top 5 Status'})

,Country of Origin,Years of Top 5 Status
0,Various/Unknown,30
1,Angola,23
2,Liberia,20
3,Dem. Rep. of the Congo,20
4,Colombia,11
5,Cuba,9
6,Haiti,7
7,Sierra Leone,6
8,Senegal,4
9,Chile,4


In [19]:
refugees_out_top3_sources['Country'].value_counts().reset_index().rename(columns={'index':'Destination Country', 'Country':'Years of Top 3 Status'}).to_csv('Processed Data/refugees_out_top3_sources_count.csv')
refugees_out_top3_sources['Country'].value_counts().reset_index().rename(columns={'index':'Destination Country', 'Country':'Years of Top 3 Status'})

refugees_out_top5_sources['Country'].value_counts().reset_index().rename(columns={'index':'Destination Country', 'Country':'Years of Top 5 Status'}).to_csv('Processed Data/refugees_out_top5_sources_count.csv')
refugees_out_top5_sources['Country'].value_counts().reset_index().rename(columns={'index':'Destination Country', 'Country':'Years of Top 5 Status'})

,Destination Country,Years of Top 5 Status
0,United States of America,24
1,Canada,23
2,Germany,15
3,Italy,15
4,Australia,14
5,Sweden,10
6,United Kingdom,6
7,Ireland,5
8,Belgium,3
9,Switzerland,3


In [20]:
refugees_out_top5_sources_count = refugees_out_top5_sources['Country'].value_counts().reset_index().rename(columns={'index':'Destination Country', 'Country':'Years of Top 5 Status'})
refugees_out_top5_sources_count

,Destination Country,Years of Top 5 Status
0,United States of America,24
1,Canada,23
2,Germany,15
3,Italy,15
4,Australia,14
5,Sweden,10
6,United Kingdom,6
7,Ireland,5
8,Belgium,3
9,Switzerland,3


In [21]:
#Some preliminary tables that help us make the final timeline by country:

refugees_in_top3_sources_pivot = refugees_in_top3_sources[['Year','Country']].copy()
refugees_in_top3_sources_pivot['Count'] = 1
refugees_in_top3_sources_pivot

refugees_out_top3_sources_pivot = refugees_out_top3_sources[['Year','Country']].copy()
refugees_out_top3_sources_pivot['Count'] = 1
refugees_out_top3_sources_pivot

refugees_in_top5_sources_pivot = refugees_in_top5_sources[['Year','Country']].copy()
refugees_in_top5_sources_pivot['Count'] = 1
refugees_in_top5_sources_pivot

refugees_out_top5_sources_pivot = refugees_out_top5_sources[['Year','Country']].copy()
refugees_out_top5_sources_pivot['Count'] = 1
refugees_out_top5_sources_pivot

,Year,Country,Count
0,1971,Chile,1
1,1972,Chile,1
2,1980,Italy,1
3,1981,Italy,1
4,1982,Italy,1
5,1983,Italy,1
6,1984,Italy,1
7,1985,Italy,1
8,1986,Italy,1
9,1987,Italy,1


In [28]:
#refugees_in_top3_by_year = refugees_in_top3_sources_pivot.pivot_table(refugees_in_top_3_sources_pivot,index=['Country'], columns='Year', values='Count')
refugees_in_top3_by_year = refugees_in_top3_sources_pivot.pivot(index='Country', columns='Year', values='Count')
#refugees_in_top3_by_year.fillna('', inplace=True)
#refugees_in_top3_by_year.replace([1], ['*'], inplace=True) 
#refugees_in_top3_by_year.to_csv('Processed Data/refugees_in_top3_by_year.csv')
#refugees_in_top3_by_year

ValueError: Index contains duplicate entries, cannot reshape

In [32]:
pd.options.display.max_columns = None 
refugees_out_top3_by_year = refugees_out_top3_sources_pivot[['Year','Country','Count']].pivot(index='Country', columns='Year', values='Count')
refugees_out_top3_by_year.fillna('', inplace=True)
refugees_out_top3_by_year.replace([1], ['*'], inplace=True) 
refugees_out_top3_by_year.to_csv('Processed Data/refugees_out_top3_by_year.csv')
refugees_out_top3_by_year

Year,1971,1972,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Australia,,,,,,,,,,,,,,,,,,,*,*,*,*,*,*,,,,,,,,,,,,,,,
Brazil,,,,,,,,,,,,,,,,,,,,,,,,,,,,*,,,,,,,,,,,
Canada,,,,,,,,,,,,,,,,,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
Chile,*,*,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Denmark,,,,,,,,,,,,*,*,,,,,,,,,,,,,,,,,,,,,,,,,,
Germany,,,,,,,,,,,,,,,,,,,,,,,,,*,*,*,,*,*,*,*,*,*,*,*,*,,
Italy,,,*,*,*,*,*,*,*,*,*,,,,,,,,,,,,,,,,,,,,,,,,,,,*,*
Mexico,,,,,,,,,,,,,,,,*,,,,,,,,,,,,,,,,,,,,,,,
Sweden,,,,,,,,,,,,,*,*,*,*,*,*,,,,,,,,,,,,,,,,,,,,,


In [30]:
refugees_in_top5_by_year = refugees_in_top5_sources_pivot[['Year','Country','Count']].pivot(index='Country', columns='Year', values='Count')
refugees_in_top5_by_year.fillna('', inplace=True)
refugees_in_top5_by_year.replace([1], ['*'], inplace=True) 
refugees_in_top5_by_year.to_csv('Processed Data/refugees_in_top5_by_year.csv')
refugees_in_top5_by_year

ValueError: Index contains duplicate entries, cannot reshape

In [33]:
pd.options.display.max_columns = None 
refugees_out_top5_by_year = refugees_out_top5_sources_pivot[['Year','Country','Count']].pivot(index='Country', columns='Year', values='Count')
refugees_out_top5_by_year.fillna('', inplace=True)
refugees_out_top5_by_year.replace([1], ['*'], inplace=True) 
refugees_out_top5_by_year.to_csv('Processed Data/refugees_out_top5_by_year.csv')
refugees_out_top5_by_year

Year,1971,1972,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Australia,,,,,,,,,,,,,,,,,,,*,*,*,*,*,*,*,*,*,*,*,*,*,,*,,,,,,
Belgium,,,,,,,,,,,,,,,,,,,,,*,*,*,,,,,,,,,,,,,,,,
Brazil,,,,,,,,,,,,,,,,,,,,,,,,,,,,*,,,,*,,,,,,,
Canada,,,,,,,,,,,,,,,,,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
Chile,*,*,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Cuba,,,,,,,,,,,,,,,,,*,,,,,,,,,,,,,,,,,,,,,,
Denmark,,,,,,,,,,,,*,*,,,,,,,,,,,,,,,,,,,,,,,,,,
France,,,,,,,,,,,,,,,,,,,,,,,,,,,,,*,,,,,,,,,,
Germany,,,,,,,,,,,,,,,,,,,,,,,,,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*


In [22]:
refugees_in_top3_by_year = refugees_in_top3_sources_pivot[['Year','Country','Count']].pivot(index='Country', columns='Year', values='Count')
refugees_in_top3_by_year.fillna('', inplace=True)
refugees_in_top3_by_year.replace([1], ['*'], inplace=True) 
refugees_in_top3_by_year.to_csv('Processed Data/refugees_in_top3_by_year.csv')
refugees_in_top3_by_year

refugees_out_top3_by_year = refugees_out_top3_sources_pivot[['Year','Country','Count']].pivot(index='Country', columns='Year', values='Count')
refugees_out_top3_by_year.fillna('', inplace=True)
refugees_out_top3_by_year.replace([1], ['*'], inplace=True) 
refugees_out_top3_by_year.to_csv('Processed Data/refugees_out_top3_by_year.csv')
refugees_out_top3_by_year

refugees_in_top5_by_year = refugees_in_top5_sources_pivot[['Year','Country','Count']].pivot(index='Country', columns='Year', values='Count')
refugees_in_top5_by_year.fillna('', inplace=True)
refugees_in_top5_by_year.replace([1], ['*'], inplace=True) 
refugees_in_top5_by_year.to_csv('Processed Data/refugees_in_top5_by_year.csv')
refugees_in_top5_by_year

refugees_out_top5_by_year = refugees_out_top5_sources_pivot[['Year','Country','Count']].pivot(index='Country', columns='Year', values='Count')
refugees_out_top5_by_year.fillna('', inplace=True)
refugees_out_top5_by_year.replace([1], ['*'], inplace=True) 
refugees_out_top5_by_year.to_csv('Processed Data/refugees_out_top5_by_year.csv')
refugees_out_top5_by_year

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
refugees_in_top3_by_year

In [ ]:
refugees_in_top5_by_year

In [ ]:
refugees_out_top3_by_year

In [ ]:
refugees_out_top5_by_year